# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe688c6a250>
├── 'a' --> tensor([[-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809]])
└── 'x' --> <FastTreeValue 0x7fe688c6a370>
    └── 'c' --> tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                        [ 0.7361, -0.0727, -0.2247,  0.3612],
                        [ 1.0653,  0.7350,  2.2299,  1.2176]])

In [4]:
t.a

tensor([[-0.9634,  0.4240, -1.3390],
        [-0.0619,  0.5273, -0.3809]])

In [5]:
%timeit t.a

63.7 ns ± 0.0584 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe688c6a250>
├── 'a' --> tensor([[0.7551, 0.6618, 0.9277],
│                   [1.1065, 1.2684, 1.5656]])
└── 'x' --> <FastTreeValue 0x7fe688c6a370>
    └── 'c' --> tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                        [ 0.7361, -0.0727, -0.2247,  0.3612],
                        [ 1.0653,  0.7350,  2.2299,  1.2176]])

In [7]:
%timeit t.a = new_value

60.6 ns ± 0.0358 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809]]),
    x: Batch(
           c: tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                      [ 0.7361, -0.0727, -0.2247,  0.3612],
                      [ 1.0653,  0.7350,  2.2299,  1.2176]]),
       ),
)

In [10]:
b.a

tensor([[-0.9634,  0.4240, -1.3390],
        [-0.0619,  0.5273, -0.3809]])

In [11]:
%timeit b.a

55.6 ns ± 0.0598 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8077, -1.1382, -0.9368],
               [ 1.5722, -0.8105,  0.8320]]),
    x: Batch(
           c: tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                      [ 0.7361, -0.0727, -0.2247,  0.3612],
                      [ 1.0653,  0.7350,  2.2299,  1.2176]]),
       ),
)

In [13]:
%timeit b.a = new_value

487 ns ± 0.0912 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

959 ns ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.7 µs ± 24.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 854 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 658 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe5d9ea1a30>
├── 'a' --> tensor([[[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]],
│           
│                   [[-0.9634,  0.4240, -1.3390],
│                    [-0.0619,  0.5273, -0.3809]]])
└── 'x' --> <FastTreeValue 0x7fe5e45cdfa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 72.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe5d9b890a0>
├── 'a' --> tensor([[-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809],
│                   [-0.9634,  0.4240, -1.3390],
│                   [-0.0619,  0.5273, -0.3809]])
└── 'x' --> <FastTreeValue 0x7fe5d9b89a60>
    └── 'c' --> tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                        [ 0.7361, -0.0727, -0.2247,  0.3612],
                 

In [23]:
%timeit t_cat(trees)

30.2 µs ± 62 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.5 µs ± 58.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]],
       
               [[-0.9634,  0.4240, -1.3390],
                [-0.0619,  0.5273, -0.3809]]]),
    x: Batch(
           c: tensor([[[ 2.3221, -0.7659, -0.3737,  0.1301],
                       [ 0.7361, -0.0727, -0.2247,  0.3612],
                       [ 1.0653,  0.7350,  2.2299,  1.2176]],
         

In [26]:
%timeit Batch.stack(batches)

78.8 µs ± 95 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809],
               [-0.9634,  0.4240, -1.3390],
               [-0.0619,  0.5273, -0.3809]]),
    x: Batch(
           c: tensor([[ 2.3221, -0.7659, -0.3737,  0.1301],
                      [ 0.7361, -0.0727, -0.2247,  0.3612],
                      [ 1.0653,  0.7350,  2.2299,  1.2176],
                      [ 2.3221, -0.7659, -0.3737,  0.1301],
                      [ 0.7361, -

In [28]:
%timeit Batch.cat(batches)

142 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 4.19 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
